In [2]:
#pip install pyside2

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 본인확인을 위한 message 창 띄우기
import sys
from PyQt5 import QtCore, QtWidgets, QtGui
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QMessageBox
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot

class MyApp(QWidget):
    def __init__(self):
        super().__init__()
        self.title = 'PyQt5'
        self.left = 100
        self.top = 100
        self.width = 600
        self.height = 200
        self.initUI()
 
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        self.move(500, 200)
        buttonReply = QMessageBox.information(
            self, '출결 시스템', "본인이 맞습니까?", 
            QMessageBox.Yes | QMessageBox.No | QMessageBox.Cancel)
 
        if buttonReply == QMessageBox.Yes:
            print('Yes clicked.')
        elif buttonReply == QMessageBox.Cancel:
            print('Cancel clicked.')
        else:
            print('No clicked.')

In [6]:
#얼굴인식 
import face_recognition
import cv2
import camera
import os
import numpy as np

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)

    def __del__(self):
        self.video.release()

    def get_frame(self):
        ret, frame = self.video.read()
        return frame

class FaceRecog():
    def __init__(self):
        self.camera = VideoCamera()

        self.known_face_encodings = []
        self.known_face_names = []

        # 이미지를 불러오기
        dirname = 'knowns'
        files = os.listdir(dirname)
        for filename in files:
            name, ext = os.path.splitext(filename)
            if ext == '.jpg':
                self.known_face_names.append(name)
                pathname = os.path.join(dirname, filename)
                # 이미지에서 얼굴 영역 추출, face landmark를 이용해 얼굴 특징 분석
                # knwon_face_encoding에 저장
                img = face_recognition.load_image_file(pathname)
                face_encoding = face_recognition.face_encodings(img)[0]
                self.known_face_encodings.append(face_encoding)

        # Initialize some variables
        self.face_locations = []
        self.face_encodings = []
        self.face_names = []
        self.process_this_frame = True

    def __del__(self):
        del self.camera

    def get_frame(self):
        # Grab a single frame of video
        frame = self.camera.get_frame()

        # 얼굴인식 속도를 빠르게 하기 위해 frame의 크기를 1/4로 줄임
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        if self.process_this_frame:
            # 현재 비디오의 frame에서 얼굴 영역과 특징 추출
            self.face_locations = face_recognition.face_locations(rgb_small_frame)
            self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

            self.face_names = []
            for face_encoding in self.face_encodings:
                # frame에서 추출한 얼굴 특징과 knowns에 있던 사진 얼굴의 특징을 비교한 거리 측정
                # 거리가 더 가까울수록(작을수록) 비슷한 얼굴로 볼 수 있음
                distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                min_value = min(distances)

                # min_value가 0.6 이상 경우 다른 사람
                name = "Unknown"
                # min_value가 0.6 미만이고 최솟값을 가질수록 일치
                if min_value < 0.6:
                    index = np.argmin(distances)
                    name = self.known_face_names[index]

                self.face_names.append(name)
        self.process_this_frame = not self.process_this_frame
        
        # 결과 화면
        for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
            # frame의 크기 다시 4배 해줌
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # 얼굴 주변 박스 그리기
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # 얼굴 밑에 라벨과 함께 이름 보여주기
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
            # 얼굴이 이미지와 같다면 attended, 아니라면 not registered 출력
            if name=="Unknown":
                cv2.putText(frame, "Not registered.", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, name+" Attended", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

        return frame

    def get_jpg_bytes(self):
        frame = self.get_frame()
        # We are using Motion JPEG, but OpenCV defaults to capture raw images,
        # so we must encode it into JPEG in order to correctly display the
        # video stream.
        ret, jpg = cv2.imencode('.jpg', frame)
        return jpg.tobytes()

if __name__ == '__main__':
    face_recog = FaceRecog()
    print(face_recog.known_face_names)
    while True:
        frame = face_recog.get_frame()

        # show the frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
    # do a bit of cleanup
    cv2.destroyAllWindows()
    
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    myapp = MyApp()
    
    print('finish')

['jh', 'jini', 'yj (2)', 'yj']
Yes clicked.
finish
